In [ ]:
from pynq import Overlay
ol = Overlay('matrix.bit')

In [1]:
#ol?
dma = ol.axi_dma_0

NameError: name 'ol' is not defined

In [ ]:
data_send = dma.sendchannel
data_recv = dma.recvchannel

In [ ]:
import numpy as np
import random

In [ ]:
np.random.seed(2)
data_A = np.random.uniform(low=0, high=5, size=(100, 100)).astype(float)
data_B = np.random.uniform(low=0, high=5, size=(100, 100)).astype(float)

In [ ]:
data_A, data_B

(array([[2.17997451, 0.12963116, 2.74831239, ..., 3.30740663, 0.85238567,
         4.40826118],
        [3.8900408 , 0.66977104, 4.34458313, ..., 2.85137105, 4.58357306,
         3.5113212 ],
        [2.50608226, 2.53043417, 1.09410393, ..., 2.1833494 , 0.07528067,
         4.46452942],
        ...,
        [4.23750542, 0.65679314, 3.74152348, ..., 3.06932209, 1.50690678,
         4.17862858],
        [2.95279362, 1.40779618, 2.35501374, ..., 3.27752982, 1.09370769,
         4.46091562],
        [3.60345567, 4.42545675, 3.94375322, ..., 3.67088066, 3.86581593,
         0.86636664]]),
 array([[0.22604679, 1.30417374, 4.28352158, ..., 1.18557389, 4.96360888,
         4.05388414],
        [0.97183793, 4.17362645, 0.755709  , ..., 4.93269947, 1.3711461 ,
         1.85736668],
        [1.6515429 , 1.80712386, 0.55670065, ..., 4.88112831, 4.37630447,
         4.41531313],
        ...,
        [2.29704164, 1.89011005, 1.07503642, ..., 2.70962298, 4.37429115,
         4.60649329],
        [0.5

In [ ]:
def multiply(matrix_a, matrix_b):
    result_matrix = np.zeros((100,100))
    for i in range(100):
        for j in range(100):
            for k in range(100):
                result_matrix[i][j] += matrix_a[i][k] * matrix_b[k][j]

    return result_matrix

In [ ]:
import time
start_time = time.time()
ans = multiply(data_A, data_B) #software
end_time = time.time()
print(end_time - start_time)

19.620131015777588


In [ ]:
from pynq import allocate

In [ ]:
data_a_f = data_A.flatten()
data_b_f = data_B.flatten()

data_input = np.concatenate([data_a_f, data_b_f])

In [ ]:
data_input

array([2.17997451, 0.12963116, 2.74831239, ..., 0.04084358, 4.35430084,
       0.19495433])

In [ ]:
input_buffer = allocate(20000, np.float32)
output_buffer = allocate((100,100), np.float32)

In [ ]:
np.copyto(input_buffer, data_input)

In [ ]:
start_time = time.time()
data_send.transfer(input_buffer)
data_recv.transfer(output_buffer)
data_send.wait()
data_recv.wait()
end_time = time.time()
print(end_time - start_time)

0.002955913543701172


In [ ]:

output_buffer, ans

(PynqBuffer([[565.00684, 602.74146, 596.5425 , ..., 580.2141 , 645.79175,
              567.85767],
             [601.51855, 609.2251 , 600.94214, ..., 601.1333 , 608.6721 ,
              613.2898 ],
             [663.9607 , 705.4531 , 702.67725, ..., 688.5664 , 713.23315,
              640.34644],
             ...,
             [582.42114, 604.6057 , 615.0408 , ..., 613.56323, 686.46606,
              531.6653 ],
             [599.4673 , 631.48755, 629.23193, ..., 593.00244, 665.62524,
              606.28394],
             [644.78   , 732.0747 , 689.333  , ..., 634.65894, 685.78955,
              672.1543 ]], dtype=float32),
 array([[565.0718379 , 602.81545775, 596.60994919, ..., 580.28372845,
         645.85884484, 567.92741414],
        [601.58686623, 609.2988807 , 601.01085809, ..., 601.20056324,
         608.74233715, 613.36037263],
        [664.03528774, 705.5370774 , 702.7523641 , ..., 688.64344565,
         713.31221197, 640.42499167],
        ...,
        [582.49075702, 604.6

In [ ]:
diff = np.dot(data_A, data_B) - output_buffer

In [ ]:
#rms error
rms_error = np.sqrt(np.mean(diff**2))

In [ ]:
diff

PynqBuffer([[0.06500196, 0.07400267, 0.06746872, ..., 0.06961712,
             0.06709679, 0.06974812],
            [0.06831154, 0.07378304, 0.06871942, ..., 0.06726246,
             0.07021801, 0.07057771],
            [0.07459438, 0.0839524 , 0.07511801, ..., 0.0770394 ,
             0.07905767, 0.07855612],
            ...,
            [0.06961444, 0.0761641 , 0.06862765, ..., 0.0656415 ,
             0.07204046, 0.06823783],
            [0.07358559, 0.07889339, 0.07438889, ..., 0.07306872,
             0.0777679 , 0.07271707],
            [0.07462524, 0.07860067, 0.07293042, ..., 0.07337875,
             0.07617038, 0.07273672]])

In [ ]:
rms_error

PynqBuffer(0.07273262)

In [ ]:
%%time

np.dot(data_A, data_B)

CPU times: user 9.93 ms, sys: 13 µs, total: 9.94 ms
Wall time: 7.66 ms


array([[565.0718379 , 602.81545775, 596.60994919, ..., 580.28372845,
        645.85884484, 567.92741414],
       [601.58686623, 609.2988807 , 601.01085809, ..., 601.20056324,
        608.74233715, 613.36037263],
       [664.03528774, 705.5370774 , 702.7523641 , ..., 688.64344565,
        713.31221197, 640.42499167],
       ...,
       [582.49075702, 604.68187699, 615.10939914, ..., 613.62887392,
        686.53810491, 531.73352103],
       [599.54087074, 631.56644222, 629.30632249, ..., 593.07551013,
        665.70301204, 606.35665261],
       [644.85465453, 732.1533077 , 689.40593824, ..., 634.73231429,
        685.86572116, 672.22703359]])